In [ ]:
# script to look at CCQEMAT output and check the effects of a 15 GeV/c cut on p||
# Heidi Schellman, November 28, 2022

from ROOT import *
from PlotUtils import *
import sys,os, time

In [ ]:
c1 = TCanvas()
c1.cd()
# what are these files?  they are the full anti-nu data sample run through the new CCQEMAT framework. They differ in the p|| cut on the muon,
# 100 GeV is the loose cut, #15 is the tight cut. 
dir2 = "$HOME/Dropbox/CCQE/val2"
dir1 = "$HOME/Dropbox/CCQE/example_v1"
MnvTunev2 = os.path.join(dir2,"val2_tuned_analyze9.root")
norm = os.path.join(dir1,"MnvTunev1_tuned_analyze9.root")


In [ ]:
f = TFile.Open(MnvTunev2,"READONLY")
g = TFile.Open(norm,"READONLY")

# these are the parent files - to check on migration
fm = TFile.Open(os.path.join(dir2,"val2.root"))
gm = TFile.Open(os.path.join(dir1,"MnvTunev1.root"))

var = "ptmu"

In [ ]:
# define the plots you want to look at and their format.
types = ["bkgsub","bkgsub_unfolded","bkgsub_unfolded_effcorr","bkgsub_unfolded_effcorr_sigma","sigmaMC"]
colors = {"bkgsub":1,"bkgsub_unfolded":2,"bkgsub_unfolded_effcorr":4,"bkgsub_unfolded_effcorr_sigma":8,"sigmaMC":6}
markers = {"bkgsub":20,"bkgsub_unfolded":21,"bkgsub_unfolded_effcorr":22,"bkgsub_unfolded_effcorr_sigma":23,"sigmaMC":24}
styles = {"bkgsub":0,"bkgsub_unfolded":0,"bkgsub_unfolded_effcorr":0,"bkgsub_unfolded_effcorr_sigma":1,"sigmaMC":7}
print ("done with setup")

In [ ]:
# read in one of the inputs and make certain it has a decent migration 
f.cd()
h_var_MnvTunev2 = {}
for type in types:
    h_var_MnvTunev2[type] = MnvH1D()
    h_var_MnvTunev2[type] = f.Get("h_QElike_"+var+"_"+type)
mig1 = MnvH2D()
mig1 = fm.Get("h___QElike___qelike___"+var+"___response_tuned_migration")
mig1.Draw("colz")
c1.Draw()

In [ ]:
g.cd()
h_var_MnvTunev1 = {}
for type in types:
    h_var_MnvTunev1[type] = MnvH1D()
    h_var_MnvTunev1[type] = g.Get("h_QElike_"+var+"_"+type)
mig2 = MnvH2D()
mig2 = gm.Get("h___QElike___qelike___"+var+"___response_tuned_migration")
mig2.Draw("colz")
c1.Draw()

In [ ]:
# loop over the types and make ratios for each level of processing

r = {}
for type in types:
    r[type] = MnvH1D()
    r[type] = h_var_MnvTunev2[type].Clone("ratio")
    opt = "B"
    if "sigma" in type:
        opt = ""
    r[type].Divide(h_var_MnvTunev2[type],h_var_MnvTunev1[type],1.,1.,opt)

In [ ]:
# now plot all of the different levels

count = 0
leg = TLegend()
leg.SetHeader("Effect of p|| cut")
gStyle.SetOptStat(0)
for type in types:
    count +=1
    r[type].GetYaxis().SetTitle("MnvTunev2/MnvTunev1")
    
    opt = "PE same"
    if count == 1:
        opt = "PE"
    if "sigma" in type:
        opt = "hist same"
        r[type].SetMarkerSize(0.5)
    r[type].SetMarkerColor(colors[type])
    r[type].SetLineColor(colors[type])
    r[type].SetMarkerStyle(markers[type])
    r[type].SetLineStyle(styles[type])
    
    r[type].SetMaximum(1.2)
    r[type].SetMinimum(0.50)
    
    leg.AddEntry(r[type],type,"lpe")
    r[type].Draw(opt)
    
    
leg.Draw()
c1.SetLogx()
c1.Draw()


In [ ]:
# conclusion, the 15 GeV cut reduces the rate by about 1% all the way across. 


In [ ]:
# now compare to Amit's cross section.  
amitDir = "$HOME/Dropbox/CCQE/v27b/ver2"

f_amit = TFile.Open(os.path.join(amitDir,"XS_q2_proj_CV_rebinned.root"),"READONLY")
amit2D = MnvH2D()
if var == "Q2QE":
    amit2D = f_amit.Get("h_enu_var_dataCrossSection").GetCVHistoWithStatError()
if "mu" in var:
    f_amit = TFile.Open(os.path.join(amitDir,"XS_pzmu_CV.root"),"READONLY")
    amit2D = f_amit.Get("h_pzmu_ptmu_dataCrossSection").GetCVHistoWithStatError()
amit1D = amit2D.ProjectionY()
if var =="pzmu": 
    amit1D = amit2D.ProjectionX()

amit1D.Scale(1.,"width")
amit1D.Print()
leg = TLegend()
leg.SetHeader("Compare to Amit")
leg.AddEntry(amit1D,"Amit","lpe")
c1.SetLogx(1)
c1.SetLogy(1)
amit1D.Draw("pe")
nocut=h_var_MnvTunev2["bkgsub_unfolded_effcorr_sigma"].GetCVHistoWithStatError()
cut=h_var_MnvTunev1["bkgsub_unfolded_effcorr_sigma"].GetCVHistoWithStatError()
leg.AddEntry(nocut,"MnvTunev2","lpe")
leg.AddEntry(cut,"MnvTunev1","lpe")
nocut.Scale(1.0,"width")
cut.SetLineColor(2)
cut.Scale(1.,"width")
nocut.Draw("hist same")
cut.Draw("hist same")
leg.Draw()
c1.Draw()


In [ ]:
c1.SetLogy(0)
r = MnvH1D()
u = MnvH1D()
m = MnvH1D()
a = MnvH1D()
r = h_var_MnvTunev1["bkgsub_unfolded_effcorr_sigma"].Clone()
u = h_var_MnvTunev2["bkgsub_unfolded_effcorr_sigma"].Clone()
r.Scale(1.,"width")
u.Scale(1.,"width")
a = amit1D.Clone()
m = h_var_MnvTunev1["sigmaMC"].Clone()
m.Scale(1.,"width")
r.Divide(r,m,1.,1.)
u.Divide(u,m,1.,1.)
a.Divide(a,m,1.,1.)
m.Divide(m,m,1.,1.)
u.SetLineColor(6)
r.SetLineColor(4)
u.SetMarkerStyle(24)
r.SetMarkerStyle(20)
r.SetMarkerColor(4)
u.SetMarkerColor(6)
r.GetYaxis().SetTitle("ratio to MnvTunev1 sigmaMC")
leg = TLegend(.2,.7,.5,.9)
leg.SetHeader("Compare to Amit")
leg.AddEntry(r,"MnvTunev1","pe")
leg.AddEntry(u,"MnvTunev2","pel")
leg.AddEntry(a,"Amit","pe")
leg.AddEntry(m,"sigmaMC","l")
r.SetMaximum(1.4)
r.SetMinimum(0.8)
r.Draw("PE")
u.Draw("PE same")
a.Draw("PE same")
m.Draw("hist same")
leg.Draw()
c1.Draw()

In [ ]:
# I was using the funky migration matrices for this so not unexpected. 